# Création de Dashbord avec Dash

## Installation de Dash

In [1]:
# !pip install dash plotly

## Importation de bibliothèques nécessaires

In [11]:
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import plotly.figure_factory as ff

## Chargement et préparation des données pour le Dashbord

### Chargement des données et néttoyage

In [12]:
# Chargement des données
df = pd.read_csv("marketing_campaign.csv", sep=";")

# Convertir 'Dt_Customer' en format datetime
df["Dt_Customer"] = pd.to_datetime(df["Dt_Customer"], format="%Y-%m-%d")

# Imputation des valeurs manquantes par la médiane
df['Income'].fillna(df['Income'].median(), inplace=True)


### Création de nouvelles colonnes

In [13]:
# Création d'une variable indiquant le nombre total de campagnes acceptées
campagne_cols = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5']
df['TotalAcceptedCampaigns'] = df[campagne_cols].sum(axis=1)

# Création d'une variable "TotalPurchases" pour les achats totaux
achat_cols = ['NumDealsPurchases', 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases']
df['TotalPurchases'] = df[achat_cols].sum(axis=1)

# Création d'une variable "TotalAmountSpent" pour les dépenses totales
depense_cols = ['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
df['TotalAmountSpent'] = df[depense_cols].sum(axis=1)

# Calcul de l'âge des clients
current_year = pd.Timestamp.now().year

# Création de la colonne Age
df['Age'] = current_year - df['Year_Birth']

# Définition des variables quantitatives
variables_quanti = ['Income', 'Recency', 'TotalPurchases', 'TotalAmountSpent', 'Age']


### Segmentation par tranche d'âge et revenu

In [14]:
# -Segmentation par âge 
# Définir des intervalles d'âge
bins_age = [0, 30, 45, 60, 100]
labels_age = ['<30', '30-45', '45-60', '60+']
df['Age_Group'] = pd.cut(df['Age'], bins=bins_age, labels=labels_age, right=False)

# Segmentation par revenu
# Utiliser des quantiles pour créer des groupes de revenus (4 groupes)
df['Income_Group'] = pd.qcut(df['Income'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])

# Création d'un DataFrame pour l'analyse par segment
# Agrégation des indicateurs par segment (Âge et Revenu)
grouped = df.groupby(['Age_Group', 'Income_Group']).agg(
    Total_Customers=('ID', 'count'),
    Avg_Accepted_Campaigns=('TotalAcceptedCampaigns', 'mean'),
    Conversion_Rate=('Response', lambda x: x.sum() / x.count() * 100),
    Total_Amount=('TotalAmountSpent', 'sum')
).reset_index()


### Calcul des indicateurs

In [15]:
# Nombre total de clients uniques
total_clients = df['ID'].nunique()

#  Calcul du taux d'acceptation par campagne
taux_par_campagne = {
    campagne: (df[campagne] > 0).sum() / total_clients * 100
    for campagne in campagne_cols
}

#  Taux d'acceptation global (au moins une campagne acceptée)
taux_global = (df['TotalAcceptedCampaigns'] > 0).sum() / total_clients * 100

#  Taux moyen d'offres acceptées par client
moyenne_acceptations = df['TotalAcceptedCampaigns'].mean()

#  Taux de conversion de la dernière campagne
taux_response = df['Response'].sum() / total_clients * 100

## Créer l'application Dash

### Initialisation de l'application Dash

In [16]:
# Initialisation de l'application
# Inclusion du thème FLATLY et de Bootstrap Icons
external_stylesheets = [
    dbc.themes.FLATLY,
    "https://cdn.jsdelivr.net/npm/bootstrap-icons@1.5.0/font/bootstrap-icons.css"
]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets, serve_locally=True)

Création de l’application Dash et ajout d’un thème Bootstrap ("Flatly") pour un design plus professionnel.

### Définir la mise en page du dashboard (Layout)

Le **Layout** en **Dash** est l’endroit où tu définis ce qui est affiché sur notre **dashboard** :
* Titres
* Graphiques
* Tableaux
* Boutons, menus déroulants, sliders...

Tout ce qui est visible dans ton tableau de bord est défini dans le Layout !

En gros c'est il définit la structure du dashboard.

In [17]:
# Construction du Layout de notre dashboard
app.layout = dbc.Container([                         #Structure principale du dashboard (Debut du layout)
    # Titre principal
    html.H1([
        html.I(className="bi bi-bar-chart-fill me-2"),
        html.Strong("Dashboard - Analyse Campagnes Marketing")
    ], className="text-center mt-4 mb-4"),
    
    # Row 0 : Cartes récapitulatives
    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardBody([
                    html.H5([
                        html.I(className="bi bi-people-fill me-2"),
                        html.Strong("Nombre total de clients")
                    ], className="card-title"),
                    html.H3(f"{total_clients}", className="text-primary")
                ])
            ], className="shadow-sm mb-4"), width=3
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardBody([
                    html.H5([
                        html.I(className="bi bi-check-circle-fill me-2"),
                        html.Strong("Taux d'acceptation global")
                    ], className="card-title"),
                    html.H3(f"{taux_global:.2f}%", className="text-success")
                ])
            ], className="shadow-sm mb-4"), width=3
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardBody([
                    html.H5([
                        html.I(className="bi bi-hand-thumbs-up-fill me-2"),
                        html.Strong("Taux moyen d'offres acceptées")
                    ], className="card-title"),
                    html.H3(f"{moyenne_acceptations:.2f}%", className="text-warning")
                ])
            ], className="shadow-sm mb-4"), width=3
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardBody([
                    html.H5([
                        html.I(className="bi bi-bar-chart-line me-2"),
                        html.Strong("Taux de conversion (dernière campagne)")
                    ], className="card-title"),
                    html.H3(f"{taux_response:.2f}%", className="text-danger")
                ])
            ], className="shadow-sm mb-4"), width=3
        ),
    ]),
    
    # Row 1 : Bar Chart pour le taux d'acceptation par campagne
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.I(className="bi bi-graph-up me-2"),
                    html.Strong("Taux d'Acceptation par Campagne")
                ]),
                dbc.CardBody([
                    dcc.Graph(id="bar-taux-campagne")
                ])
            ], className="shadow-sm mb-4")
        ], width=12),
    ]),
    
    # Row 2 : Histogramme et Scatterplot interactif
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.I(className="bi bi-bar-chart me-2"),
                    html.Strong("Distribution des variables")
                ]),
                dbc.CardBody([
                    dcc.Dropdown(
                        id='var-dropdown',
                        options=[{'label': var, 'value': var} for var in variables_quanti],
                        value='Income',
                        clearable=False,
                        className="mb-3"
                    ),
                    dcc.Graph(id='histogram-plot')
                ])
            ], className="shadow-sm mb-4")
        ], width=6),
        
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.I(className="bi bi-arrows-move me-2"),
                    html.Strong("Comparaison entre deux variables")
                ]),
                dbc.CardBody([
                    html.Label("Variable X :", className="mt-2"),
                    dcc.Dropdown(
                        id='scatter-x-dropdown',
                        options=[{'label': var, 'value': var} for var in variables_quanti],
                        value='Income',
                        clearable=False,
                        className="mb-2"
                    ),
                    html.Label("Variable Y :", className="mt-2"),
                    dcc.Dropdown(
                        id='scatter-y-dropdown',
                        options=[{'label': var, 'value': var} for var in ['TotalPurchases', 'TotalAmountSpent', 'Age']],
                        value='TotalPurchases',
                        clearable=False,
                        className="mb-2"
                    ),
                    dcc.Graph(id='scatter-plot')
                ])
            ], className="shadow-sm mb-4")
        ], width=6),
    ]),

    # Row 3 : Boxplot des dépenses vs campagnes acceptées
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.I(className="bi bi-currency-dollar me-2"),
                    html.Strong("Dépenses vs Campagnes Acceptées")
                ]),
                dbc.CardBody([
                    dcc.Graph(id='boxplot-plot')
                ])
            ], className="shadow-sm mb-4")
        ], width=12),
    ]),
    
    # Row 4 : Barplot des segments (conversion)
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.I(className="bi bi-pie-chart me-2"),
                    html.Strong("Taux de conversion par segment (Âge et Revenu)")
                ]),
                dbc.CardBody([
                    dcc.Graph(id='bar-conversion-segment')
                ])
            ], className="shadow-sm mb-4")
        ], width=12),
    ]),
    
    # Row 5 : Barplot des segments (dépense)
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader([
                    html.I(className="bi bi-cash-stack me-2"),
                    html.Strong("Montant total dépensé par segment (Âge et Revenu)")
                ]),
                dbc.CardBody([
                    dcc.Graph(id='bar-depense-segment')
                ])
            ], className="shadow-sm mb-4")
        ], width=12),
    ]),
], fluid=True)  #Fin du layout

**Inclusion de Bootstrap Icons :**
Le lien "https://cdn.jsdelivr.net/npm/bootstrap-icons@1.5.0/font/bootstrap-icons.css" est ajouté dans external_stylesheets afin de charger les icônes.

Sélection d’icônes :

    Nombre total de clients : bi-people-fill
    Taux d'acceptation global : bi-check-circle-fill
    Taux moyen d'offres acceptées : bi-hand-thumbs-up-fill
    Taux de conversion : bi-bar-chart-line
    Taux d'acceptation par campagne : bi-graph-up
    Distribution des variables : bi-bar-chart
    Comparaison entre deux variables : bi-arrows-move
    Dépenses vs Campagnes acceptées : bi-currency-dollar
    Taux de conversion par segment : bi-pie-chart
    Montant total dépensé par segment : bi-cash-stack
    
    "me-2" pour espacer l'icône du texte

 ### Callbacks pour les taux et Graphiques

Un **callback** est une fonction qui permet de **mettre à jour** un élément du **dashboard** en fonction d’un **événement utilisateur**. Les callbacks permettent de rendre **interactif** le **Dashbord**.

In [18]:
# Callbacks pour les graphiques
@app.callback(
    Output("bar-taux-campagne", "figure"),
    Input("var-dropdown", "value")
)
# Bar Chart pour le taux d'acceptation par campagne
def update_bar_taux(_):
    # Création d'un DataFrame pour stocker les taux d'acceptation par campagne
    taux_campagne_df = pd.DataFrame({
        "Campagne": campagne_cols,
        "Taux (%)": [round((df[campagne] > 0).sum() / total_clients * 100, 2) for campagne in campagne_cols]
    })
    # Dictionnaire de mapping pour renommer les campagnes
    mapping = {
        'AcceptedCmp1': 'Campagne 1',
        'AcceptedCmp2': 'Campagne 2',
        'AcceptedCmp3': 'Campagne 3',
        'AcceptedCmp4': 'Campagne 4',
        'AcceptedCmp5': 'Campagne 5'
    }
    # Remplacer les valeurs dans la colonne "Campagne"
    taux_campagne_df['Campagne'] = taux_campagne_df['Campagne'].replace(mapping)
    # Création du graphique
    fig = px.bar(
        taux_campagne_df, 
        x="Campagne", 
        y="Taux (%)", 
        text="Taux (%)",
        color="Taux (%)",
        color_continuous_scale="sunset"
    )
    # Personnalisation des étiquettes
    fig.update_traces(
        texttemplate="%{text:.2f}%",
        textposition="outside",
        marker=dict(line=dict(color="black", width=0.8))
    )
    # Mise en page du graphique
    fig.update_layout(
        title="Taux d'Acceptation par Campagne",
        xaxis_title="Campagnes",
        yaxis_title="Taux (%)",
        coloraxis_colorbar_title="Taux (%)",
        paper_bgcolor="#f8f9fa",    #code couleurs
        plot_bgcolor="#f8f9fa",
        font=dict(family="Arial", size=14)
    )
    return fig

@app.callback(
    Output('histogram-plot', 'figure'),
    Input('var-dropdown', 'value')
)
# Histogramme dynamique
def update_histogram(selected_var):
    fig = px.histogram(
        df, 
        x=selected_var, 
        nbins=30, 
        title=f"Distribution de {selected_var}",
        color_discrete_sequence=['#1f77b4']
    )
    fig.update_layout(bargap=0.1, paper_bgcolor="#f8f9fa", plot_bgcolor="#f8f9fa")
    return fig

@app.callback(
    Output('scatter-plot', 'figure'),
    [Input('scatter-x-dropdown', 'value'),
     Input('scatter-y-dropdown', 'value')]
)
# Scatterplot interactif
def update_scatter(selected_x, selected_y):
    fig = px.scatter(
        df, 
        x=selected_x, 
        y=selected_y, 
        title=f"Relation entre {selected_x} et {selected_y}",
        color_discrete_sequence=['#e377c2']
    )
    fig.update_layout(paper_bgcolor="#f8f9fa", plot_bgcolor="#f8f9fa")
    return fig

@app.callback(
    Output('boxplot-plot', 'figure'),
    Input('var-dropdown', 'value')
)
# Boxplot : TotalAmountSpent vs TotalAcceptedCampaigns
def update_boxplot(_):
    # Remplacement des valeurs numériques par des labels plus explicites
    df['Campagne_Acceptee'] = df['TotalAcceptedCampaigns'].map({
        0: "Campagne 1",
        1: "Campagne 2",
        2: "Campagne 3",
        3: "Campagne 4",
        4: "Campagne 5"
    })
    # Création du boxplot
    fig = px.box(
        df, 
        x='Campagne_Acceptee', 
        y='TotalAmountSpent', 
        title="Montant total dépensé vs Campagnes acceptées",
        color='Campagne_Acceptee',
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    # Mise à jour des axes et mise en forme
    fig.update_layout(
        xaxis_title="Campagnes acceptées",
        yaxis_title="Montant total dépensé par client",
        legend_title="Campagnes acceptées",
        paper_bgcolor="#f8f9fa",
        plot_bgcolor="#f8f9fa"
    )
    return fig

@app.callback(
    Output('bar-conversion-segment', 'figure'),
    Input('var-dropdown', 'value')
)
# Barplot des segments (conversion)
def update_barplot(_):
    fig = px.bar(
        grouped, 
        x="Age_Group", 
        y="Conversion_Rate", 
        color="Income_Group", 
        barmode="group",
        title="Taux de Conversion par Segment (Âge et Revenu)",
        labels={"Age_Group": "Groupe d'âge", "Conversion_Rate": "Taux de Conversion (%)",
                "Income_Group": "Groupe de revenu"},
        color_discrete_sequence=px.colors.qualitative.Set1
    )
    fig.update_layout(paper_bgcolor="#f8f9fa", plot_bgcolor="#f8f9fa")
    return fig

@app.callback(
    Output('bar-depense-segment', 'figure'),
    Input('var-dropdown', 'value')
)
# Barplot des segments (dépense)
def update_bar_depense(_):
    fig = px.bar(
        grouped, 
        x="Age_Group", 
        y="Total_Amount", 
        color="Income_Group", 
        barmode="group",
        title="Montant Total Dépensé par Segment (Âge et Revenu)",
        labels={"Age_Group": "Groupe d'âge", "Total_Amount": "Montant total dépensé",
                "Income_Group": "Groupe de revenu"},
        color_discrete_sequence=px.colors.qualitative.Set2
    )
    fig.update_layout(paper_bgcolor="#f8f9fa", plot_bgcolor="#f8f9fa")
    return fig


### Lancement de l’application

In [19]:
# Lancer l'application
if __name__ == '__main__':
    app.run_server(debug=True)
# lance l’application web sur un serveur local